# Connect to OMERO with Templated Variables

In [ ]:
import omero.clients
from omero.gateway import BlitzGateway

import matplotlib.pyplot as plt
import ipywidgets as widgets

In [ ]:
import pprint #pourquoi?
from getpass import getpass
hostname = {% if omero_hostname %} {{ omero_hostname }} {% else %} "omero.epfl.ch" {% endif %}
hostport = {% if omero_hostport %} {{ omero_hostport }} {% else %} 4064 {% endif %}
username = {% if omero_username %} {{ omero_username }} {% else %} input("Username: ") {% endif %} 
conn = BlitzGateway(
    username, getpass("OMERO Password: "),
    host=hostname, port=hostport, secure=True)
conn.connect()  

# Choose your group, project and dataset of interest

In [47]:
group_list = []
for g in conn.getGroupsMemberOf():
    group_list = group_list + [(g.getName(), g.getId())]
    
print(group_list)

[('system', 0), ('ptbiop', 253), ('public-data', 305), ('OMERO parade (public)', 304), ('BIOP Public', 403)]


In [52]:
defaultgroup = conn.getGroupFromContext()
groupdropdwn = widgets.Dropdown(
    options = group_list,
    value = defaultgroup.getId(),
    description='Group:',
)
display(groupdropdwn)

Dropdown(description='Group:', index=1, options=(('system', 0), ('ptbiop', 253), ('public-data', 305), ('OMERO…

In [54]:
my_exp_id = conn.getUser().getId()
project_list=[]
#can remove 'owner': my_exp_id 'group': groupdropdwn.value,
print(groupdropdwn.value)
for project in conn.getObjects("Project", opts={'owner': my_exp_id,
                                            'order_by': 'lower(obj.name)', 'offset': 0}):
   
    project_list = project_list + [(project.getName(),project.getId())]
print(project_list)

305
[('Baptiste data', 1051), ('BigDataViewerTest', 756), ('BIOP Microscopes', 567), ('Demo Project', 467), ('HDF5 test', 602), ('IDR0021', 527), ('Margaux Duchamp vsi', 579), ('MetaMorph Stack (STK)', 758), ('Operetta Archive test', 604), ('Qupath test', 538), ('Test_3D', 524), ('Tiff Virtual Stack', 1004)]


In [40]:
projectdropdwn = widgets.Dropdown(
    options = project_list,
    description='Project:',
)
display(projectdropdwn)

Dropdown(description='Project:', options=(('Baptiste data', 1051), ('BigDataViewerTest', 756), ('BIOP Microsco…

In [7]:
dataset_list=[]
project = conn.getObject("Project", projectdropdwn.value)
    # We can get Datasets with listChildren, since we have the Project already.
    # Or conn.getObjects("Dataset", opts={'project', id}) if we have Project ID
for dataset in project.listChildren():
    dataset_list=dataset_list + [(dataset.getName(),dataset.getId())]

In [8]:
datasetdropdwn = widgets.Dropdown(
    options = dataset_list,
    description='Dataset:',
)
display(datasetdropdwn)

Dropdown(description='Dataset:', options=(('CEP120', 735), ('CEP152', 736), ('CEP192-M', 737), ('CPAP', 738), …

In [9]:
imageIDlist = []
for image in conn.getObjects('Image', opts={'dataset': datasetdropdwn.value}):
    imageIDlist = imageIDlist + [image.getId()]


In [10]:
def display_image(imageID):
    image = conn.getObject("Image", imageID)
    z = 0
    t = 0
    renderedImage = image.renderImage(z, t)
    fig = plt.figure()
    plt.imshow(renderedImage)

In [11]:
a = widgets.SelectionSlider(
    options=imageIDlist,
    description='Image ID',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

out = widgets.interactive_output(display_image, {'imageID': a})

display(a, out)


SelectionSlider(continuous_update=False, description='Image ID', options=(5872, 5871, 5870, 5869, 5868, 5867, …

Output()